## NMF = Not Monday night Football !

In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
from random import randint
import numpy as np
from sklearn.decomposition import NMF
import pandas as pd
import random
from matplotlib import pyplot as plt
%matplotlib inline
import csv

# User Input

In [377]:
#importing ratings and movies csv files
PATH2 = "ratings.csv"
PATH3 = "movies.csv"
ratings, movies_ind = pd.read_csv(PATH2), pd.read_csv(PATH3)

In [354]:
# create an empty array the length of number of movies in system
user_ratings = np.zeros(9724)

In [378]:
#format ratings dataframe
del ratings['timestamp']
ratings.set_index(['userId','movieId'], inplace=True)
ratings = ratings.unstack(0)

In [288]:
ratings_count = ratings.count(axis=1) #count the number of ratings for each movie as a measure of popularity
top = pd.DataFrame(ratings_count.sort_values(ascending = False).head(10)) #create a dataframe of the top 20 most popular movies

In [289]:
top.reset_index(inplace=True)

In [290]:
movies_ind.set_index('movieId',inplace=True)

In [296]:
top_movies_g = movies_ind.loc[top['movieId']]['title'].values

## Of the following movies, rate all that you have seen on a scale of 1-5. 
## If you have not seen a movie, rate 0.

In [370]:
#creates a list of ratings for the prompted movies
user_input = []
for i in range(0,10):
    answer = int(input("How would you rate " + str(top_movies_g[i])))
    user_input.append(answer)

How would you rate Forrest Gump (1994)2
How would you rate Shawshank Redemption, The (1994)3
How would you rate Pulp Fiction (1994)4
How would you rate Silence of the Lambs, The (1991)5
How would you rate Matrix, The (1999)6
How would you rate Star Wars: Episode IV - A New Hope (1977)4
How would you rate Jurassic Park (1993)3
How would you rate Braveheart (1995)4
How would you rate Terminator 2: Judgment Day (1991)3
How would you rate Schindler's List (1993)4


In [371]:
movies_ind.reset_index(inplace=True)

In [372]:
top_movies_index = movies_ind.index[top['movieId']].values

In [373]:
top_movies_index

array([ 356,  318,  296,  593, 2571,  260,  480,  110,  589,  527])

In [374]:
# inputs user rating into large array (9,000+ count) at appropriate indexes
for i in range(0,10):
    user_ratings[top_movies_index[i]] = user_input[i]

# NMF Modeling

In [379]:
PATH4 = "movies.csv"
movies_ind = pd.read_csv(PATH4)

ratings = ratings.fillna(0)
ratings = ratings["rating"]
ratings = ratings.transpose()

In [380]:
ratings.head(2)

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [381]:
R = pd.DataFrame(ratings)
# model assumes R ~ PQ'
model = NMF(n_components=5, init='random', random_state=10)
model.fit(R)

P = model.components_  # Movie feature
Q = model.transform(R)  # User features

In [382]:
query = user_ratings.reshape(1,-1)

In [383]:
t=model.transform(query)

In [384]:
outcome = np.dot(t,P)

In [385]:
outcome=pd.DataFrame(outcome)

In [386]:
outcome = outcome.transpose()

In [387]:
outcome['movieId'] = movies_ind['movieId']

In [388]:
outcome = outcome.rename(columns={0:'rating'})

In [389]:
outcome

,rating,movieId
0,0.049066,1
1,0.036427,2
2,0.019165,3
3,0.003525,4
4,0.017535,5
5,0.037087,6
6,0.021666,7
7,0.004575,8
8,0.005220,9
9,0.051139,10


In [169]:
for x in top.index:
    if x == 

Int64Index([ 277,  257,  314,  510,   97,  507,  418,  398,  123,  461,   43,
              46, 1938,  508, 2224,  337,  509,  224,   31,  322,  334,  138,
             506,    0,  615, 1502,  307,  659,  520,  134,  897, 3633, 4791,
             275,  546,  325,  254,  395,  910, 4131, 2144,  253,    9,  126,
             899,  512, 3136,  378,  302,   32],
           dtype='int64')

In [139]:
top =outcome.sort_values(by='rating',ascending=False).head(50)

top_movies = []
for x in top['movieId']:
    top_movies.append(movies_ind[movies_ind['movieId']== x])

top_movies

[     movieId                             title       genres
 277      318  Shawshank Redemption, The (1994)  Crime|Drama,
      movieId                title                       genres
 257      296  Pulp Fiction (1994)  Comedy|Crime|Drama|Thriller,
      movieId                title                    genres
 314      356  Forrest Gump (1994)  Comedy|Drama|Romance|War,
      movieId                             title                 genres
 510      593  Silence of the Lambs, The (1991)  Crime|Horror|Thriller,
     movieId              title            genres
 97      110  Braveheart (1995)  Action|Drama|War,
      movieId                              title         genres
 507      589  Terminator 2: Judgment Day (1991)  Action|Sci-Fi,
      movieId                 title                            genres
 418      480  Jurassic Park (1993)  Action|Adventure|Sci-Fi|Thriller,
      movieId                 title    genres
 398      457  Fugitive, The (1993)  Thriller,
      movieId     

# Selecting a Movie

In [140]:
# top_movies = pd.DataFrame(top_movies)

In [148]:
top_movies_g[0]

,movieId,title,genres
314,356,Forrest Gump (1994),Comedy|Drama|Romance|War


In [162]:
top_movies[0].shape

(1, 3)

In [165]:
top_movies[0]['movieId']

277    318
Name: movieId, dtype: int64

In [151]:
for x in top_movies:
    if top_movies == top_movies_g:
        print('repeat')

In [159]:
top_movies[0]

,movieId,title,genres
277,318,"Shawshank Redemption, The (1994)",Crime|Drama


In [29]:
#importing genres
PATH4 = "movie_genres_years.csv"
movie_genres = pd.read_csv(PATH4)

In [30]:
genres_columns = movie_genres.columns[3:23]

In [31]:
# select a random movie from top five options
# we should filter by genre next
Select = top_movies[0:5][randint(0, 4)]

In [32]:
Select

,movieId,title,genres
257,296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller


In [33]:
outcome.sort_values(by='rating',ascending=False)

,rating,movieId
277,1.229559,318
257,1.181794,296
314,1.164490,356
510,1.010917,593
97,1.007184,110
507,0.838561,589
418,0.816432,480
398,0.791485,457
123,0.788366,150
461,0.785229,527
